In [2]:
#load libraries
import ee
import datetime
import fiona
from shapely.geometry import shape
import itertools
import os.path
import time
import os

In [3]:
#set important variables that probably do change from time to time
extent_shapefile_path = '/home/dan/Documents/react_data/Cities_React/Boundaries.shp'
modis_product = 'MODIS/006/MOD09A1'
product = 'MOD09A1'
start_year = 2004
end_year = 2016
end_year = end_year + 1 #because how range works
variables = ['sur_refl_b01','sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07','QA','StateQA']
target_crs = 'SR-ORG:6974'

In [4]:
#load the shapefile
boundaries = fiona.open(extent_shapefile_path)
the_cities = list(boundaries.items())

In [5]:
#get a list of the city names
citynames = [item[1]['properties']['Name'] for item in the_cities]

In [6]:
#create a grid/dictionary of things to download
listolists = [variables, citynames, range(start_year,end_year)]
combos = list(itertools.product(*listolists))

In [7]:
#find which files are already downloaded
dlpath = '/modis/'
filenames = [dlpath + c[1].replace(' ','') + "_" + product +'_'+c[0]+'_'+str(c[2]) +'.tif'  for c in combos]
#already_downloaded = [os.path.isfile(fff) for fff in filenames]

In [8]:
#Start earth engine
ee.Initialize()
#pull the product
imgs = ee.ImageCollection(modis_product)

In [9]:
#Read in the most recent list of files
with open('/media/dan/googledrive/modis_file_list.txt', 'r') as f:
    gdrivefilelist = f.read().splitlines() 

In [10]:
check = 0
for ccc in range(0, len(combos)):
    #check to see if the file is already downloaded
    if (filenames[ccc] in gdrivefilelist) == False:
        check = check +1
print check

660


In [11]:
#for each possible combo
counter = 0
for ccc in range(0, len(combos)):
    #check to see if the file is already downloaded
    if (filenames[ccc] in gdrivefilelist) == False:
        year = combos[ccc][2]
        city_name_space = combos[ccc][1]
        city_name = city_name_space.replace(' ','')
        vari = combos[ccc][0]
        
        #sort out the year
        start = str(year)+'-'+'01-01'
        end = str(year)+'-'+'12-31'
        dat = imgs.filterDate(start, end).select(vari).toList(1000);

        #find the bounding box of the city
        for cit in range(0,len(the_cities)):
            if city_name_space == the_cities[cit][1]['properties']['Name']:
                bbox = shape(the_cities[cit][1]['geometry']).bounds
            

        city_name = city_name.replace(' ','')

        #name the job
        job_id = city_name + "_" + product +'_'+vari+'_'+str(year)

        result = ee.Image.cat(dat.get(0))

        #create a polygon to clip things too
        # xMin, yMin, xMax, yMax
        target_projection = ee.Projection(target_crs)
        thecity = ee.Geometry.Rectangle(bbox) #.transform(target_projection,250)

        #convert an image collection into an image
        #also, save the dates for layer names later (on the first layer)
        #if theiter == 0:
        #    namefile = open('/media/dan/googledrive/' + product + '.txt', "w")
        #    namefile.write("%s\n" % result.get('system:index').getInfo())
        
        for i in range(1,dat.length().getInfo()):
            newimage = ee.Image.cat(dat.get(i))
            
            #if iter == 0:
            #    namefile.write("%s\n" % newimage.get('system:index').getInfo())
            
            result = ee.Image.cat([result, newimage])
        #if iter == 0:
        #    namefile.close()
        #    iter = 1


        #result = result.clip(thecity)
        #request to download
        #https://stackoverflow.com/questions/39219705/how-to-download-images-using-google-earth-engines-python-api
        task = ee.batch.Export.image.toDrive(image = result, folder = 'modis', 
                                             description = job_id, fileNamePrefix = job_id, region = thecity.getInfo().get('coordinates'))
        #print(job_id)
        task.start()
        
        
        
        if (counter +1) % 100 == 0:
            time.sleep(80)
        
        counter = counter + 1

0

In [ ]:
print year, city_name, vari

In [ ]:
task.start

In [ ]:
end

In [ ]:
job_id

In [ ]:
namefile = open('/media/dan/googledrive/' + job_id + '.txt', "w")
for item in names:
  namefile.write("%s\n" % item)
namefile.close()

In [ ]:
newimage.get('system:index').getInfo()

In [ ]:
namefile.close()